# Tensor Parallelism

# Prepare data

$$
y = xw
$$

x(n, dim)

w(dim, 2dim)

y(n, 2dim)

y_label(n, 2dim)

gradient(dim, 2dim)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
bs = 2 
row = dim = 4
col = out_dim = dim * 2
x = torch.arange(bs * dim, dtype = torch.float32).reshape(bs, dim) 
w = torch.arange(dim * out_dim, dtype = torch.float32, requires_grad = True).reshape(dim, out_dim)
print(x)
print(w)

tensor([[0., 1., 2., 3.],
        [4., 5., 6., 7.]])

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.],
        [16., 17., 18., 19., 20., 21., 22., 23.],
        [24., 25., 26., 27., 28., 29., 30., 31.]], grad_fn=<ViewBackward0>)

In [3]:
y_label = torch.randn(bs, out_dim)
y_pred = torch.randn(bs, out_dim)

# pytorch gradient

先采用pytorch自动求导计算梯度。可以用来检验我们手动求导的准确性

In [4]:
w.retain_grad()
w.grad = None
mse_loss = nn.MSELoss(reduction='mean')

y_pred = x @ w
loss_torch = mse_loss(y_pred, y_label)
loss_torch.backward()
print(loss_torch)
print(w.grad)

tensor(82850.6484, grad_fn=<MseLossBackward0>)

tensor([[151.8721, 162.4626, 173.4557, 185.0397, 195.8915, 207.5966, 218.1116,
         229.2710],
        [203.9184, 217.9152, 232.3963, 247.5970, 261.8807, 277.3813, 291.0889,
         305.9322],
        [255.9646, 273.3677, 291.3369, 310.1544, 327.8700, 347.1660, 364.0662,
         382.5934],
        [308.0109, 328.8203, 350.2776, 372.7117, 393.8593, 416.9507, 437.0434,
         459.2547]])

# tensor gradient

$$
\begin{align}
y &= xw \\
\mathcal{l} &=||y'-y||^2 = ||y'-xw||^2 \\
w &= \arg\min_{w}||y'-xw||^2 \\
\frac{\partial l}{\partial w} &=\frac{\partial l}{\partial y}\frac{\partial y}{\partial w} =  2x^T(y'-xw)\\
\end{align}
$$

In [5]:
y_pred = x @ w
delta_y = y_pred - y_label

In [6]:
delta_w = x.t() @ delta_y / out_dim # dim
print(delta_w)

tensor([[151.8721, 162.4626, 173.4557, 185.0397, 195.8915, 207.5966, 218.1116,
         229.2710],
        [203.9184, 217.9152, 232.3963, 247.5970, 261.8807, 277.3813, 291.0889,
         305.9322],
        [255.9646, 273.3677, 291.3369, 310.1544, 327.8700, 347.1660, 364.0662,
         382.5934],
        [308.0109, 328.8203, 350.2776, 372.7117, 393.8593, 416.9507, 437.0434,
         459.2547]], grad_fn=<DivBackward0>)

In [7]:
w.retain_grad()
w.grad = None
mse_loss = nn.MSELoss(reduction='mean')

y_pred = x @ w
loss_torch = mse_loss(y_pred, y_label)
loss_torch.backward()
print(loss_torch)
print(w.grad)

tensor(82850.6484, grad_fn=<MseLossBackward0>)

tensor([[151.8721, 162.4626, 173.4557, 185.0397, 195.8915, 207.5966, 218.1116,
         229.2710],
        [203.9184, 217.9152, 232.3963, 247.5970, 261.8807, 277.3813, 291.0889,
         305.9322],
        [255.9646, 273.3677, 291.3369, 310.1544, 327.8700, 347.1660, 364.0662,
         382.5934],
        [308.0109, 328.8203, 350.2776, 372.7117, 393.8593, 416.9507, 437.0434,
         459.2547]])

# Tensor Parallel

In [8]:
import copy
# w.grad = None
w_row = torch.arange(dim * out_dim, dtype = torch.float32).reshape(dim, out_dim)
w_col = torch.arange(dim * out_dim, dtype = torch.float32).reshape(dim, out_dim)

## row-tensor parallel

In [9]:
row_dim = dim // 2
w_row_1 = w_row[:row_dim, :]
w_row_2 = w_row[row_dim:, :]

print(w_row_1.shape)
print(w_row_2.shape)
print(w_row_1)
print(w_row_2)

torch.Size([2, 8])

torch.Size([2, 8])

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

tensor([[16., 17., 18., 19., 20., 21., 22., 23.],
        [24., 25., 26., 27., 28., 29., 30., 31.]])

In [17]:
x_col_1 = x[:, :row_dim]
x_col_2 = x[:, row_dim:]
y_1 = x[:, :row_dim] @ w_row_1
y_2 = x[:, row_dim:] @ w_row_2
print(y_1.shape)
print(y_2.shape)
print((y_1+y_2).shape)

torch.Size([2, 8])

torch.Size([2, 8])

torch.Size([2, 8])

In [11]:
delta_y = (y_1 + y_2 - y_label)

grad_row_1 = x_col_1.t() @ delta_y 
grad_row_2 = x_col_2.t() @ delta_y 

print(grad_row_1.shape)
print(grad_row_2.shape)

torch.Size([2, 8])

torch.Size([2, 8])

In [12]:
grad_row = torch.cat((grad_row_1,grad_row_2), dim = 0) / out_dim
print(grad_row)

tensor([[151.8721, 162.4626, 173.4557, 185.0397, 195.8915, 207.5966, 218.1116,
         229.2710],
        [203.9184, 217.9152, 232.3963, 247.5970, 261.8807, 277.3813, 291.0889,
         305.9322],
        [255.9646, 273.3677, 291.3369, 310.1544, 327.8700, 347.1660, 364.0662,
         382.5934],
        [308.0109, 328.8203, 350.2776, 372.7117, 393.8593, 416.9507, 437.0434,
         459.2547]])

## col-tensor parallel

In [13]:
col_dim = out_dim // 2
w_col_1 = w_col[: , :col_dim]
w_col_2 = w_col[: , col_dim:]

print(w_col_1.shape)
print(w_col_2.shape)
print(w_col_1)
print(w_col_2)

torch.Size([4, 4])

torch.Size([4, 4])

tensor([[ 0.,  1.,  2.,  3.],
        [ 8.,  9., 10., 11.],
        [16., 17., 18., 19.],
        [24., 25., 26., 27.]])

tensor([[ 4.,  5.,  6.,  7.],
        [12., 13., 14., 15.],
        [20., 21., 22., 23.],
        [28., 29., 30., 31.]])

In [14]:
y_1 = x @ w_col_1
y_2 = x @ w_col_2
print(y_1.shape)
print(y_2.shape)

torch.Size([2, 4])

torch.Size([2, 4])

In [15]:
y_1_delta = y_1 - y_label[:, col_dim:]
y_2_delta = y_2 - y_label[:, :col_dim]
print(y_1_delta.shape)
print(y_2_delta.shape)

torch.Size([2, 4])

torch.Size([2, 4])

In [16]:
grad_col_1 = x.t() @ y_1_delta
grad_col_2 = x.t() @ y_2_delta 
grad_col = torch.cat( (grad_col_1, grad_col_2), dim = 1) / out_dim
print(grad_col_1.shape)
print(grad_col_2.shape)
print(grad_col.shape)
print(grad_col_1)
print(grad_col_2)
print(grad_col)

torch.Size([4, 4])

torch.Size([4, 4])

torch.Size([4, 8])

tensor([[1215.1318, 1308.7727, 1392.8927, 1482.1680],
        [1631.0459, 1755.0504, 1864.7109, 1983.4578],
        [2046.9600, 2201.3281, 2336.5293, 2484.7476],
        [2462.8740, 2647.6060, 2808.3474, 2986.0376]])

tensor([[1566.9767, 1651.7007, 1739.6460, 1832.3176],
        [2095.3469, 2207.3213, 2323.1709, 2444.7761],
        [2623.7170, 2762.9419, 2906.6958, 3057.2349],
        [3152.0872, 3318.5625, 3490.2205, 3669.6934]])

tensor([[151.8915, 163.5966, 174.1116, 185.2710, 195.8721, 206.4626, 217.4557,
         229.0397],
        [203.8807, 219.3813, 233.0889, 247.9322, 261.9184, 275.9152, 290.3964,
         305.5970],
        [255.8700, 275.1660, 292.0662, 310.5934, 327.9646, 345.3677, 363.3370,
         382.1544],
        [307.8593, 330.9507, 351.0434, 373.2547, 394.0109, 414.8203, 436.2776,
         458.7117]])